In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import explained_variance_score, r2_score
from sklearn.model_selection import cross_validate
import tensorflow as tf
from sklearn.metrics import mean_squared_error as MSE
from pathlib import Path
from sklearn.metrics import classification_report
import xgboost as xg 
from xgboost import XGBClassifier
import pandas as pd , numpy as np , datetime, os
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers
import seaborn as sns,  matplotlib.pyplot as plt, joblib as jlib
import tensorflow as tf, keras_tuner, tensorflow.keras as keras
from tensorflow.keras import layers



In [2]:
pip install csv-to-sqlite

Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv_to_sqlite

In [4]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="UTF-8")
#change path to "Resources/insurance.csv" for jupyter notebook
input_files = ['Resources/insurance.csv']
#change path to "Resources/insurance.sqlite" for jupyter notebook
csv_to_sqlite.write_csv(input_files, "Resources/insurance.sqlite", options)


Error on table Resources/insurance.csv: 
 table [insurance] already exists
Written 0 rows into 1 tables in 0.014 seconds


0

In [5]:
import sqlalchemy
from sqlalchemy import create_engine, text, inspect, func
import warnings
engine = create_engine("sqlite:///Resources/insurance.sqlite")

#gives warning but easier to get the dataframe from querying
warnings.filterwarnings('ignore')
conn = engine.raw_connection()
insurance_df = pd.read_sql_query(sql=f"SELECT age ,sex, bmi, children, smoker , region , ROUND(charges,2) as charges FROM insurance", con=conn)

In [6]:
insurance_df

age     sex     bmi  children smoker     region   charges
0      19  female  27.900         0    yes  southwest  16884.92
1      18    male  33.770         1     no  southeast   1725.55
2      28    male  33.000         3     no  southeast   4449.46
3      33    male  22.705         0     no  northwest  21984.47
4      32    male  28.880         0     no  northwest   3866.86
...   ...     ...     ...       ...    ...        ...       ...
1333   50    male  30.970         3     no  northwest  10600.55
1334   18  female  31.920         0     no  northeast   2205.98
1335   18  female  36.850         0     no  southeast   1629.83
1336   21  female  25.800         0     no  southwest   2007.95
1337   61  female  29.070         0    yes  northwest  29141.36

[1338 rows x 7 columns]

In [7]:
insurance_df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [8]:
# @title OneHot + MinMax
df1 = insurance_df.copy()

col_to_encode = ['sex', 'smoker', 'region']

X_OneHot_model = OneHotEncoder(sparse_output=False)
X_OH_fit = X_OneHot_model.fit(insurance_df[col_to_encode])


df_encoded =  pd.DataFrame(X_OneHot_model.transform(insurance_df[col_to_encode]), columns=X_OneHot_model.get_feature_names_out())
df1 = df1.drop(col_to_encode, axis=1).merge(df_encoded, left_index=True, right_index=True)


In [10]:
df1.head()

age     bmi  children   charges  sex_female  sex_male  smoker_no  \
0   19  27.900         0  16884.92         1.0       0.0        0.0   
1   18  33.770         1   1725.55         0.0       1.0        1.0   
2   28  33.000         3   4449.46         0.0       1.0        1.0   
3   33  22.705         0  21984.47         0.0       1.0        1.0   
4   32  28.880         0   3866.86         0.0       1.0        1.0   

   smoker_yes  region_northeast  region_northwest  region_southeast  \
0         1.0               0.0               0.0               0.0   
1         0.0               0.0               0.0               1.0   
2         0.0               0.0               0.0               1.0   
3         0.0               0.0               1.0               0.0   
4         0.0               0.0               1.0               0.0   

   region_southwest  
0               1.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0

In [11]:
X_min_max_scaler = MinMaxScaler()
X_fit_scaler = X_min_max_scaler.fit(df1[['age','bmi','children']])
df1[['age','bmi','children']] = X_fit_scaler.transform(df1[['age','bmi','children']])


In [12]:
X = df1.drop('charges', axis = 1)
X.head(5)

age       bmi  children  sex_female  sex_male  smoker_no  smoker_yes  \
0  0.021739  0.321227       0.0         1.0       0.0        0.0         1.0   
1  0.000000  0.479150       0.2         0.0       1.0        1.0         0.0   
2  0.217391  0.458434       0.6         0.0       1.0        1.0         0.0   
3  0.326087  0.181464       0.0         0.0       1.0        1.0         0.0   
4  0.304348  0.347592       0.0         0.0       1.0        1.0         0.0   

   region_northeast  region_northwest  region_southeast  region_southwest  
0               0.0               0.0               0.0               1.0  
1               0.0               0.0               1.0               0.0  
2               0.0               0.0               1.0               0.0  
3               0.0               1.0               0.0               0.0  
4               0.0               1.0               0.0               0.0

In [13]:
y_min_max_scaler = MinMaxScaler()
y_fit_scaler = y_min_max_scaler.fit(df1[['charges']])
y = y_fit_scaler.transform(df1[['charges']])
y.ravel().reshape(1,-1)

array([[0.25161073, 0.00963598, 0.05311519, ..., 0.00810809, 0.01414366,
        0.44724875]])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 123)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1003, 11) (335, 11) (1003, 1) (335, 1)


In [15]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 2, 8)):
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=12, max_value=144, step=12),
                activation=hp.Choice("activation", ["relu", "tanh", "sigmoid", "elu"]),
                input_dim = X_train.shape[1]
            )
        )

    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))

    model.add(layers.Dense(1, activation="linear"))

    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss = "mean_absolute_error",
        metrics=['mean_absolute_error']
    )

    return model

build_model(keras_tuner.HyperParameters())

In [16]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=keras_tuner.Objective("val_mean_absolute_error", direction="min"),
    max_trials=3,
    executions_per_trial=3,
    tune_new_entries=True,
    allow_new_entries=True,
    max_retries_per_trial=5,
    max_consecutive_failed_trials=3,
    overwrite=True,
    directory="ins_dir",
    project_name="Tuning Insrance Pred Model",
    seed = 42
)

In [17]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 12, 'max_value': 144, 'step': 12, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 12, 'max_value': 144, 'step': 12, 'sampling': 'linear'}
dropout (Boolean)
{'default': False, 'conditions': []}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
!rm -rf ./logs/
%load_ext tensorboard

In [18]:
logdir_tune = os.path.join("logs", "logs_tune", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback_tune = tf.keras.callbacks.TensorBoard(logdir_tune, histogram_freq=1, write_images=True, update_freq='epoch')

In [19]:
logdir_train = os.path.join("logs", "logs_train", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback_train = tf.keras.callbacks.TensorBoard(logdir_train, histogram_freq=1, write_images=True, update_freq='epoch')

In [22]:
import time
start_time = time.time()
tuner.search(X_train, y_train,
             epochs=4,
             validation_split = 0.15,
             use_multiprocessing=True,
             verbose = 2,
             callbacks=[tensorboard_callback_tune])

print('>>> total time spent for searching :: ', time.time() - start_time, 'seconds')

>>> total time spent for searching ::  0.0014681816101074219 seconds


In [23]:
best_models = tuner.get_best_models()[0]
best_models.build(X_train.shape)

In [24]:
best_models.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (1003, 11)                0         
                                                                 
 dense (Dense)               (1003, 36)                432       
                                                                 
 dense_1 (Dense)             (1003, 72)                2664      
                                                                 
 dense_2 (Dense)             (1003, 72)                5256      
                                                                 
 dense_3 (Dense)             (1003, 60)                4380      
                                                                 
 dense_4 (Dense)             (1003, 12)                732       
                                                                 
 dense_5 (Dense)             (1003, 12)                1

In [ ]:
%tensorboard --logdir logs/logs_tune

In [25]:
tuner.results_summary()

Results summary
Results in ins_dir/Tuning Insrance Pred Model
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 2 summary
Hyperparameters:
num_layers: 6
units_0: 36
activation: tanh
units_1: 72
dropout: True
learning_rate: 0.0007767449713530701
units_2: 72
units_3: 60
units_4: 12
units_5: 12
Score: 0.0711052268743515

Trial 1 summary
Hyperparameters:
num_layers: 5
units_0: 12
activation: tanh
units_1: 36
dropout: True
learning_rate: 0.001715074355925934
units_2: 132
units_3: 24
units_4: 144
units_5: 12
Score: 0.07506890594959259

Trial 0 summary
Hyperparameters:
num_layers: 6
units_0: 12
activation: tanh
units_1: 48
dropout: True
learning_rate: 0.000505466655750672
units_2: 12
units_3: 12
units_4: 12
units_5: 12
Score: 0.08923016985257466


In [26]:
best_models.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (1003, 11)                0         
                                                                 
 dense (Dense)               (1003, 36)                432       
                                                                 
 dense_1 (Dense)             (1003, 72)                2664      
                                                                 
 dense_2 (Dense)             (1003, 72)                5256      
                                                                 
 dense_3 (Dense)             (1003, 60)                4380      
                                                                 
 dense_4 (Dense)             (1003, 12)                732       
                                                                 
 dense_5 (Dense)             (1003, 12)                1

In [ ]:
!rm -rf logs/logs_train/

In [27]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='mean_absolute_error',
    min_delta=0,
    patience=20,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [28]:
best_models.fit(X_train, y_train,
                epochs=500,
                batch_size=128,
                validation_split=0.15,
                shuffle = True,
                verbose = 1,
                callbacks=[early_stopping, tensorboard_callback_train])

Epoch 1/500
7/7 [==============================] - 0s 17ms/step - loss: 0.0753 - mean_absolute_error: 0.0753 - val_loss: 0.0719 - val_mean_absolute_error: 0.0719
Epoch 2/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0696 - mean_absolute_error: 0.0696 - val_loss: 0.0665 - val_mean_absolute_error: 0.0665
Epoch 3/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0672 - mean_absolute_error: 0.0672 - val_loss: 0.0655 - val_mean_absolute_error: 0.0655
Epoch 4/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0667 - mean_absolute_error: 0.0667 - val_loss: 0.0713 - val_mean_absolute_error: 0.0713
Epoch 5/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0682 - mean_absolute_error: 0.0682 - val_loss: 0.0720 - val_mean_absolute_error: 0.0720
Epoch 6/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0709 - mean_absolute_error: 0.0709 - val_loss: 0.0643 - val_mean_absolute_error: 0.0643
Epoch 7/500
7/7 [================

7/7 [==============================] - 0s 7ms/step - loss: 0.0554 - mean_absolute_error: 0.0554 - val_loss: 0.0454 - val_mean_absolute_error: 0.0454
Epoch 52/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0519 - mean_absolute_error: 0.0519 - val_loss: 0.0503 - val_mean_absolute_error: 0.0503
Epoch 53/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0557 - mean_absolute_error: 0.0557 - val_loss: 0.0515 - val_mean_absolute_error: 0.0515
Epoch 54/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0586 - mean_absolute_error: 0.0586 - val_loss: 0.0517 - val_mean_absolute_error: 0.0517
Epoch 55/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0534 - mean_absolute_error: 0.0534 - val_loss: 0.0460 - val_mean_absolute_error: 0.0460
Epoch 56/500
7/7 [==============================] - 0s 7ms/step - loss: 0.0524 - mean_absolute_error: 0.0524 - val_loss: 0.0491 - val_mean_absolute_error: 0.0491
Epoch 57/500
7/7 [=======================

7/7 [==============================] - 0s 8ms/step - loss: 0.0490 - mean_absolute_error: 0.0490 - val_loss: 0.0428 - val_mean_absolute_error: 0.0428
Epoch 102/500
7/7 [==============================] - 0s 9ms/step - loss: 0.0456 - mean_absolute_error: 0.0456 - val_loss: 0.0434 - val_mean_absolute_error: 0.0434
Epoch 103/500
7/7 [==============================] - 0s 8ms/step - loss: 0.0487 - mean_absolute_error: 0.0487 - val_loss: 0.0428 - val_mean_absolute_error: 0.0428
Epoch 104/500
7/7 [==============================] - 0s 8ms/step - loss: 0.0470 - mean_absolute_error: 0.0470 - val_loss: 0.0419 - val_mean_absolute_error: 0.0419
Epoch 105/500
7/7 [==============================] - 0s 8ms/step - loss: 0.0465 - mean_absolute_error: 0.0465 - val_loss: 0.0456 - val_mean_absolute_error: 0.0456
Epoch 106/500
7/7 [==============================] - 0s 8ms/step - loss: 0.0495 - mean_absolute_error: 0.0495 - val_loss: 0.0447 - val_mean_absolute_error: 0.0447
Epoch 107/500
7/7 [=================

In [ ]:
%tensorboard --logdir logs/logs_train

In [29]:
# @title Evaluation on Test Data { display-mode: "both" }

best_models.compile(loss='mean_absolute_error', optimizer='adam', metrics=[keras.metrics.R2Score()])
te_eval = best_models.evaluate(X_test, y_test, verbose = 0)
print (f'>>> Mean ABS Error: {te_eval[0]} \n>>> R2 Score: {te_eval[1]}')

>>> Mean ABS Error: 0.03742477297782898 
>>> R2 Score: 0.8401862978935242


In [30]:
# @title Recheck on Train Data { display-mode: "both" }
best_models.compile(loss='mean_absolute_error', optimizer='adam', metrics=[keras.metrics.R2Score()])
tr_eval = best_models.evaluate(X_train, y_train, verbose = 0)
print (f'>>> Mean ABS Error: {tr_eval[0]} \n>>> R2 Score: {tr_eval[1]}')

>>> Mean ABS Error: 0.042029645293951035 
>>> R2 Score: 0.7888541221618652


In [31]:
MAE_scaled = np.asarray(tr_eval[0]).reshape(1, -1)
print(f'Mean Absolute Error from any Quote:: ${y_min_max_scaler.inverse_transform(MAE_scaled).ravel()[0]:,}')

Mean Absolute Error from any Quote:: $3,754.966754976809


In [32]:
!pip install visualkeras

In [33]:
import visualkeras
visualkeras.layered_view(best_models)

In [34]:
# @title Save Model / Scaler / Encoder { display-mode: "both" }
import time
if os.path.exists('./Trained Models/') == False:
  os.mkdir ('./Trained Models')

  time_out_lim = 30
  periodic_check = 5
  start = time.time()

  while time.time() - start < time_out_lim:
    print("Time lapse:", time.time() - start)
    if os.path.exists('./Trained Models/'):
      best_models.save('./Trained Models/tuned_insurance_model.keras', overwrite=True)
      jlib.dump(X_OH_fit, r'./Trained Models/oneHotModel.pkl')
      jlib.dump(X_fit_scaler, r'./Trained Models/X_min_max_fit.pkl')
      jlib.dump(y_fit_scaler, r'./Trained Models/y_min_max_fit.pkl')
      print("Folder created...", os.path.exists('./Trained Models/') )
      break
    else:
      print ("Unable to create Trained Models folder, Timed Out...")
    time.sleep(periodic_check)



Time lapse: 3.600120544433594e-05
Folder created... True


In [35]:
X_min_max_model = jlib.load(r'./Trained Models/X_min_max_fit.pkl')
y_min_max_model = jlib.load(r'./Trained Models/y_min_max_fit.pkl')
X_oneHot_model = jlib.load(r'./Trained Models/oneHotModel.pkl')
pred_model = keras.saving.load_model('./Trained Models/tuned_insurance_model.keras')
pred_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 11)                0         
                                                                 
 dense (Dense)               (None, 36)                432       
                                                                 
 dense_1 (Dense)             (None, 72)                2664      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 60)                4380      
                                                                 
 dense_4 (Dense)             (None, 12)                732       
                                                                 
 dense_5 (Dense)             (None, 12)                1

In [36]:
def predict_ins_charges(X_raw_input_arr, y_true = None):

    feature_cols = ['age','sex', 'bmi', 'children',	'smoker',	'region']
    # X_raw_input_arr = [19, "female", 27.90, 0 , "yes", "southwest"]
    input_df = pd.DataFrame([dict(zip(feature_cols, X_raw_input_arr))])

    cols_to_encode = ['sex', 'smoker', 'region']
    one_hot_encoded_df =  pd.DataFrame(X_oneHot_model.transform(input_df[cols_to_encode]), columns=X_oneHot_model.get_feature_names_out())

    col_to_scale = ['age', 'bmi', 'children']
    input_df[col_to_scale] = X_min_max_model.transform(input_df[col_to_scale])


    input_df = input_df.drop(cols_to_encode, axis = 1).merge(one_hot_encoded_df, left_index=True, right_index=True)


    y_pred_scaled = pred_model.predict(input_df, verbose = 1)
    ISR = y_min_max_model.inverse_transform(y_pred_scaled).ravel()[0]

    if y_true:
      absErr = y_true - ISR
      percErr =  absErr / y_true * 100

      # return {"Insurance Quote": ISR,
      #         "True Value" : y_true,
      #         "Quote Error Amount": absErr,
      #         "Quote Error Percentage": percErr}

      print('>>> Insurance Quote: ${:,.2f}\n>>> True Value : ${:,.2f}\n>>> Quote Error Amount: ${:,.2f}\n>>> Quote Error Percentage: {:,.2f}%'.format(ISR, y_true, absErr, percErr))

    else:
      print(">>> Insurance Quote: ${:,.2f}".format(ISR))
      # return f">>> ISR: ${ISR:,}"

In [37]:
user_input = [28, "male", 33, 3 , "no", "southeast"]
predict_ins_charges(user_input, 4449.4620)

1/1 [==============================] - 0s 52ms/step
>>> Insurance Quote: $3,764.41
>>> True Value : $4,449.46
>>> Quote Error Amount: $685.06
>>> Quote Error Percentage: 15.40%


In [38]:
# 64, 18
Age = widgets.IntSlider(
    value=20,
    min=10,
    max=80,
    step=1,
    description='Age',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# 53.13, 15.96
BMI = widgets.FloatSlider(
    value=28,
    min=0,
    max=10.0,
    step=0.1,
    description='BMI',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# 5, 0
Children = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Children',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

Sex = widgets.Dropdown(
    options=['Female', 'Male'],
    value='Female',
    description='Gender',
    disabled=False,
)

Smoking = widgets.RadioButtons(
    options=['Yes', 'No'],
    value='No',
    description='Smoker?',
    disabled=False
)

Region = widgets.Dropdown(
    options=['Southwest', 'Southeast', 'Northwest', 'Northeast'],
    value='Southwest',
    description='Region',
    disabled=False,
)

Y_True = widgets.Text(
    value='Hello World',
    placeholder = '0',
    description ='Truly Paid',
    disabled = False
)

box_out = widgets.VBox([Age, BMI, Children, Sex, Smoking, Region, Y_True])

def f(Age, BMI, Children, Sex, Smoking, Region, Y_True):
  # [28, "male", 33, 3 , "no", "southeast"]
  if Y_True <= 0:
    Y_True = None
  else:
    Y_True = Y_True

    # print((Age, Sex, BMI, Children, Smoking, Region, Y_True))
    pred_model()

out = widgets.interactive_output(f, {'Age': Age, 'BMI': BMI, 'Children': Children,"Sex": Sex, "Smoking": Smoking, "Region": Region, 'Y_True': Y_True})

display(box_out, out)

NameError: name 'widgets' is not defined